# 1. Download Modal and Login

In [2]:
import kagglehub
kagglehub.login()

In [ ]:
ai_mathematical_olympiad_progress_prize_2_path = kagglehub.competition_download('ai-mathematical-olympiad-progress-prize-2')
#mpware_vllm_0_7_1_path = kagglehub.utility_script_install('mpware/vllm-0-7-1')
deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_1_5b_2_path = kagglehub.model_download('deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-1.5b/2')
deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_7b_2_path = kagglehub.model_download('deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-7b/2')

print('Data source import complete.')


In [13]:
!ls ~/.cache/kagglehub/

competitions models


In [14]:
# Define paths manually using your existing cached locations
ai_mathematical_olympiad_progress_prize_2_path = "/Users/sanjeeb/.cache/kagglehub/competitions/ai-mathematical-olympiad-progress-prize-2"
deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_1_5b_2_path = "/Users/sanjeeb/.cache/kagglehub/models/deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-1.5b/2"
deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_7b_2_path = "/Users/sanjeeb/.cache/kagglehub/models/deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-7b/2"

print("Using cached data and models:")
print("Competition data:", ai_mathematical_olympiad_progress_prize_2_path)
print("1.5B model:", deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_1_5b_2_path)
print("7B model:", deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_7b_2_path)

Using cached data and models:
Competition data: /Users/sanjeeb/.cache/kagglehub/competitions/ai-mathematical-olympiad-progress-prize-2
1.5B model: /Users/sanjeeb/.cache/kagglehub/models/deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-1.5b/2
7B model: /Users/sanjeeb/.cache/kagglehub/models/deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-7b/2


In [ ]:
import torch
import os
import pandas as pd
import numpy as np
import polars as pl

#import kaggle_evaluation.aimo_2_inference_server

Setting the Cutoff times as per the competiton rules

In [17]:
import time
start_time = time.time()
cutoff_time = start_time + (4 * 60 + 45) * 60
cutoff_times = [int(x) for x in np.linspace(cutoff_time, start_time + 60 * 60, 50 + 1)]

# 2. Reading the data

In [21]:
df = pd.read_csv(f"{ai_mathematical_olympiad_progress_prize_2_path}/reference.csv")
df_test = pd.read_csv(f"{ai_mathematical_olympiad_progress_prize_2_path}/test.csv")
sample_submission = pd.read_csv(f"{ai_mathematical_olympiad_progress_prize_2_path}/sample_submission.csv")

In [22]:
df.head(5)

,id,problem,answer
0,057f8a,Three airline companies operate flights from D...,79
1,192e23,Fred and George take part in a tennis tourname...,250
2,1acac0,Triangle $ABC$ has side length $AB = 120$ and ...,180
3,1fce4b,Find the three-digit number $n$ such that writ...,143
4,349493,"We call a sequence $a_1, a_2, \ldots$ of non-n...",3


In [23]:
test=df["problem"][0]
test

'Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?'

In [24]:
df_test.head(5)

,id,problem
0,000aaa,What is $1-1$?
1,111bbb,What is $0\times10$?
2,222ccc,Solve $4+x=4$ for $x$.


# 3.  Define the model path

In [ ]:
# Define the model path
model_path = deepseek_ai_deepseek_r1_transformers_deepseek_r1_distill_qwen_1_5b_2_path

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()

In [26]:
MAX_NUM_SEQS = 128
MAX_MODEL_LEN = 8192 * 3 // 2

Installing VLLM in MacOS
* git clone https://github.com/vllm-project/vllm.git
* cd vllm
Direct installation - might work on some systems without additional flags
* pip install -e .
If direct installation fails with CUDA errors, use these environment variables
to bypass CUDA requirements (optional but often necessary):
* export VLLM_TARGET_DEVICE=cpu
* export VLLM_BUILD_WITH_CUDA=0
* pip install -e .

In [32]:
import torch

# Check if MPS (Metal) is available
print(torch.backends.mps.is_available())  # Should return True on M1/M2 Macs
print(torch.backends.mps.is_built())      # True if PyTorch supports MPS

if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Apple Metal GPU
    print(f"Using device: {device}")
else:
    device = torch.device("cpu")
    print("No GPU acceleration available (MPS not supported)")

True
True
Using device: mps


In [35]:
# Convert the model (run from inside llama.cpp)
! python convert.py --input /Users/sanjeeb/.cache/kagglehub/models/deepseek-ai/deepseek-r1/Transformers/deepseek-r1-distill-qwen-1.5b/2 --outtype q4_0 --outfile deepseek-1.5b.gguf

python: can't open file '/Users/sanjeeb/Desktop/Harbin Institute of Technology/Artificial Intelligence/AI_Codes/Pre-Train-Language-Model/convert.py': [Errno 2] No such file or directory


# 4.  Loading the LLM Model with vLLM


This code initializes an LLM model using vLLM with specific configurations, such as setting the data type to float16, defining the maximum number of sequences, context length, GPU memory utilization, and enabling remote code trust for downloading the model and tokenizer.

In [ ]:
from vllm import LLM, SamplingParams
# Load the model
llm = LLM(model_path,dtype="float16",max_num_seqs=MAX_NUM_SEQS,   # Maximum number of sequences per iteration. Default is 256
    max_model_len=MAX_MODEL_LEN, # Model context length
    trust_remote_code=True,      # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=1,      # The number of GPUs to use for distributed execution with tensor parallelism
    gpu_memory_utilization=0.95, # The ratio (between 0 and 1) of GPU memory to reserve for the model
    seed=2024,)

# 5. Prompt Formatting for Math Problem Solving


This function structures the input prompt to enforce strict formatting rules for an LLM solving math problems. It ensures that the model outputs only the final numeric answer inside \boxed{} without any explanations or intermediate steps.

In [ ]:
def format_prompt(problem_text):
    return (
        "Solve the following math problem exactly.\n"
        "Return ONLY the final numeric answer inside \\boxed{}.\n"
        "THIS IS NOT A TRICK QUESTION. DON'T OVERTHINK"
        "DO NOT include explanations, reasoning, or intermediate steps.\n"
        "DO NOT repeat or rephrase the question.\n"
        "DO NOT output anything except \\boxed{ANSWER}.\n"
        "Provide the final numeric answer inside \\boxed{} only \n\n"
        "Problem: " + problem_text
    )



In [ ]:
 max_tokens = MAX_MODEL_LEN
if time.time() > cutoff_times[-1]:
    print("Speedrun")
    max_tokens = 2 * MAX_MODEL_LEN // 3

# 6. Sampling Parameters for Math Problem Solving

In [ ]:
sampling_params = SamplingParams(
    temperature=1.0,      # Forces deterministic, precise answers
    top_p=1.0,           # Consider all likely tokens (not limiting)
    top_k=-1,            # No top-k restriction
    max_tokens=max_tokens,       # Small max tokens to prevent long reasoning
    repetition_penalty=1.1,  # Discourage repeated phrases
    ignore_eos=False,    # Let generation stop naturally
)

**This Will Generate the full Response Using Transformer**

In [ ]:
# def solve_math_problem_full_response(problem_text):
#     # Tokenize input
#     inputs = tokenizer(problem_text, return_tensors="pt").to("cuda") 
    
#     # Generate response
#     with torch.no_grad():
#         output = model.generate(**inputs, max_new_tokens=700,  # Increased max_new_tokens
#                         eos_token_id=tokenizer.eos_token_id,
#                         temperature=0.9,  # Higher temperature
#                         top_p=0.85,       # Adjusted top_p
#                         num_beams=6,       # More beams
#                         repetition_penalty=1.2) # Penalize repetitions     
    
#     # Decode the response
#     response = tokenizer.decode(output[0], skip_special_tokens=True)
    
#     return response

**Response Using Transformer**

In [ ]:
# question = "what is 10+5"
# answer = solve_math_problem_full_response(question)
# print("Generated Answer:", answer)

# 7.  Generating a Math Solution Using vLLM. 

Formats the problem prompt to enforce strict answer formatting inside \boxed{}.
Uses vLLM to generate responses based on predefined sampling parameters.
Prints the generated output while ensuring no intermediate steps or explanations.

In [ ]:
problem_text = "What is 10 * 5"
formatted_prompt = format_prompt(problem_text)
# Generate response
outputs = llm.generate(formatted_prompt,sampling_params)
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, \nGenerated text: {generated_text!r}")

# 8. To calculate the Modulo

In [ ]:
def compute_final_answer(result):
    try:
        result = int(result)  # Convert to integer
    except ValueError:
        raise ValueError(f"Invalid input: {result} is not a number")

    # Apply modulo 1000 only if result is greater than 999 or negative
    if result > 999 or result < 0:
        final_answer = result % 1000
        # Ensure positive modulo result
        final_answer = final_answer if final_answer >= 0 else final_answer + 1000
    else:
        final_answer = result

    return final_answer  # Returning as an integer


In [ ]:
compute_final_answer(1023)

# 9. Extracting Final Answer from boxed response

In [ ]:
import re

def extract_boxed_text(text: str) -> str:
    pattern = r'boxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""

In [ ]:
final_answer= extract_boxed_text(generated_text)
print("Final Generated Answer:",final_answer )


In [ ]:
compute_final_answer(final_answer)

**For Only Final Boxed Answer**

In [ ]:
# import re

# def extract_boxed_text(text: str) -> str:
#     pattern = r'boxed{(.*?)}'
#     matches = re.findall(pattern, text)
#     if not matches:
#         return ""
#     for match in matches[::-1]:
#         if match != "":
#             return match
#     return ""

# def solve_math_problem(problem_text):
#     formatted_prompt = (
#     "Solve the following math problem exactly. Do not approximate. "
#     "Return ONLY the correct final numeric answer inside \\boxed{}. "
#     "Do NOT include explanations, reasoning, or intermediate steps. "
#     "If the answer is negative, still use \\boxed{}. "
#     "Ensure the answer is 100% correct before returning. "
#     "Problem: " + problem_text
# )



#     inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

#     with torch.no_grad():
#         output = model.generate(**inputs, max_new_tokens=700,  
#                         eos_token_id=tokenizer.eos_token_id,
#                         temperature=0.9, 
#                         top_p=0.85,      
#                         num_beams=6,     
#                         repetition_penalty=1.2)
#     response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

#     return extract_boxed_text(response)


In [ ]:
# question = "what is 10+5"
# answer = solve_math_problem(question)
# print("Generated Answer:", answer)

# 10.  Method that combines prompt formatting, model generation, and extracting the final answer

In [ ]:
import re

def solve_math_problem_vllm(llm, problem_text, sampling_params):
    """
    Generates a response using the LLM, extracts the final boxed answer, and returns it.
    
    Args:
        llm: The language model instance.
        problem_text (str): The math problem to solve.
        sampling_params: Parameters for the model generation.

    Returns:
        str: Extracted final numeric answer from \boxed{}.
    """

    # Format the prompt
    formatted_prompt = format_prompt(problem_text)

    # Generate response from the model
    outputs = llm.generate(formatted_prompt, sampling_params)

    # Extract generated text
    generated_text = outputs[0].outputs[0].text if outputs else ""

    # Extract the boxed answer
    final_answer = extract_boxed_text(generated_text)
    # Modulo 
    prediction_modulo=compute_final_answer(final_answer)  #Take the Modulo 

    # Print results
    print(f"Prompt: {formatted_prompt!r}\n")
    print(f"Generated text: {generated_text!r}\n")
    print(f"Final Generated Answer: {prediction_modulo}")
    return prediction_modulo


In [ ]:
df_test["problem"][0]

In [ ]:
problem_text = df_test["problem"][0]
final_answer = solve_math_problem_vllm(llm, problem_text, sampling_params)


In [ ]:

# Function to process all rows and generate predictions using Pandas
def generate_submission(df_test):
    """Generate predictions for all rows in df_test and save to submission.csv."""
    results = []

    for _, row in df_test.iterrows():  # Iterate through Pandas DataFrame rows
        id_ = row["id"]
        question = row["problem"]

        print(f"Processing ID: {id_}, Question: {question}")

        # Generate prediction using LLM
        prediction = solve_math_problem_vllm(llm, question, sampling_params)

        # Append result to list
        results.append({"id": id_, "answer": prediction})

    # Convert results to Pandas DataFrame
    submission_df = pd.DataFrame(results)

    # Save as CSV
    submission_df.to_csv("submission.csv", index=False)
    print("Submission saved as submission.csv")

# Call the function
# generate_submission(df_test)


In [ ]:
generate_submission(df_test)

In [ ]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
import polars as pl
import pandas as pd

def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # Unpack values
    id_ = id_.item(0)
    print("------")
    print(id_)
    
    question = question.item(0)
    print(question)
    # Generate prediction using the model
    prediction = solve_math_problem_vllm(llm, question, sampling_params)  # Get boxed answer
        
    print("------\n\n\n")

    print("Final Predicted Answer is",prediction)
         
    return pl.DataFrame({'id': [id_], 'answer': prediction})


In [ ]:
sample_submission.head(5)

In [ ]:
df_test.head(5)

In [ ]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [ ]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            "/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv",
            # "reference.csv",
        )
    )